<h1><center>Pawpular : PetFinder + W&B</center></h1>
                                                      
<center><img src = "https://scontent-hel3-1.xx.fbcdn.net/v/t1.6435-9/95999292_3051385531573716_8920255664832380928_n.png?_nc_cat=111&ccb=1-5&_nc_sid=6e5ad9&_nc_ohc=64hmGZb8bXAAX_AoecS&_nc_ht=scontent-hel3-1.xx&oh=6964e3c4b5d29fc4fea6f3fdd9fd1f67&oe=61A85FDF" width = "750" height = "500"/></center>                                                                                               

<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:lightgreen; border:0; color:white' role="tab" aria-controls="home"><center>Contents</center></h2>

1. [Competition Overview](#competition-overview)  
2. [Libraries](#libraries)  
3. [Weights and Biases](#weights-and-biases)   
4. [CFG](#global-config)
5. [Load Datasets](#load-datasets)  
6. [EDA](#tabular-exploration) 
7. [Conclusion about meta data](#conclusion)
8. [t-SNE | Clusters | Plots](#distribution-plots)
9. [Tables in W&B](#tableswb)
8. [Work in progress](#eof )

<a id="competition-overview"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:lightgreen; border:0; color:white' role="tab" aria-controls="home"><center>Competition Overview</center></h2>

## **<span style="color:lightgreen;">Description</span>**

In this competition, you’ll analyze raw images and metadata to predict the “Pawpularity” of pet photos.   
  
You'll train and test your model on PetFinder.my's thousands of pet profiles. Winning versions will offer accurate recommendations that will improve animal welfare.
  
If successful, your solution will be adapted into AI tools that will guide shelters and rescuers around the world to improve the appeal of their pet profiles, automatically enhancing photo quality and recommending composition improvements.   
  
As a result, stray dogs and cats can find their "furever" homes much faster. With a little assistance from the Kaggle community, many precious lives could be saved and more happy families created.

---

## **<span style="color:lightgreen;">Evaluation Criteria</span>**

Submissions are scored on the **Root mean squared error**. 

<a id="libraries"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:lightgreen; border:0; color:white' role="tab" aria-controls="home"><center>Libraries</center></h2>

In [ ]:
!pip install -q timm albumentations wandb
!pip install pandas-profiling[notebook] -q
!pip install autoviz -q

In [ ]:
import gc
import os
import glob
import sys
import cv2
import imageio
import joblib
import math
import random
import wandb
import math

import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport

from scipy.stats import kstest

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from statsmodels.graphics.gofplots import qqplot

plt.rcParams.update({'font.size': 18})
plt.style.use('fivethirtyeight')

import seaborn as sns
import matplotlib

from termcolor import colored

from multiprocessing import cpu_count
from tqdm.notebook import tqdm
from sklearn.model_selection import StratifiedKFold
from scipy.stats import pearsonr

from scipy.stats import skew
from sklearn.decomposition import PCA 
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE 
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import timm
import torch
import transformers
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import r2_score, mean_squared_error

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint

from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize
)
from albumentations.pytorch import ToTensorV2

import warnings
warnings.simplefilter('ignore')

# Wandb Login
import wandb
from kaggle_secrets import UserSecretsClient
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning import Trainer


#Importing the class function
from autoviz.AutoViz_Class import AutoViz_Class

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
sns.color_palette('pastel')

tqdm.pandas()

In [ ]:
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb-api")
wandb.login(key=secret_value_0)
run = wandb.init(project="PetFinder", entity="gusevski")


wandb_logger = WandbLogger(project="PetFinder")

<a id="weights-and-biases"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:lightgreen; border:0; color:white' role="tab" aria-controls="home"><center>Weights and Biases</center></h2>

<center><img src = "https://camo.githubusercontent.com/dd842f7b0be57140e68b2ab9cb007992acd131c48284eaf6b1aca758bfea358b/68747470733a2f2f692e696d6775722e636f6d2f52557469567a482e706e67"/></center>  

**Weights & Biases** is the machine learning platform for developers to build better models faster. 

You can use W&B's lightweight, interoperable tools to 
- quickly track experiments, 
- version and iterate on datasets, 
- evaluate model performance, 
- reproduce models, 
- visualize results and spot regressions, 
- and share findings with colleagues. 

Set up W&B in 5 minutes, then quickly iterate on your machine learning pipeline with the confidence that your datasets and models are tracked and versioned in a reliable system of record.

<a id="global-config"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:lightgreen; border:0; color:white' role="tab" aria-controls="home"><center>CFG</center></h2>

In [ ]:
class CFG:
    TRAIN_CSV_PATH = '../input/petfinder-pawpularity-score/train.csv'
    TEST_CSV_PATH = '../input/petfinder-pawpularity-score/test.csv'
    sample_submission = '../input/petfinder-pawpularity-score/sample_submission.csv'
    SEED=2021

In [ ]:
def set_seed(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
set_seed(CFG.SEED)

<a id="load-datasets"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:lightgreen; border:0; color:white' role="tab" aria-controls="home"><center>Load Datasets</center></h2>

## **<span style="color:lightgreen;">Understanding the Structure of the Dataset</span>**

> ### **<span style="color:lightgreen;">Goal of Competition</span>**
> In this competition, your task is to predict engagement with a pet's profile based on the photograph for that profile. You are also provided with hand-labelled metadata for each photo. The dataset for this competition therefore comprises both images and tabular data.
> 
> ### **<span style="color:lightgreen;">How Pawpularity Score Is Derived</span>**
> The Pawpularity Score is derived from each pet profile's page view statistics at the listing pages, using an algorithm that normalizes the traffic data across different pages, platforms (web & mobile) and various metrics.  
> Duplicate clicks, crawler bot accesses and sponsored profiles are excluded from the analysis.
>
>---

> ### **<span style="color:lightgreen;">Training Data</span>**
> - train/ - Folder containing training set photos of the form {id}.jpg, where {id} is a unique Pet Profile ID.
> - train.csv - Metadata (described below) for each photo in the training set as well as the target, the photo's Pawpularity score. The Id column gives the photo's unique Pet Profile ID corresponding the photo's file name.
> 
> ### **<span style="color:lightgreen;">Example Test Data</span>**
> In addition to the training data, we include some randomly generated example test data to help you author submission code. When your submitted notebook is scored, this example data will be replaced by the actual test data (including the sample submission).
> 
> - test/ - Folder containing randomly generated images in a format similar to the training set photos. The actual test data comprises about 6800 pet photos similar to the training set photos.
> - test.csv - Randomly generated metadata similar to the training set metadata.
> - sample_submission.csv - A sample submission file in the correct format.
>
>---

> ### **<span style="color:lightgreen;">Photo Metadata</span>**
> The train.csv and test.csv files contain metadata for photos in the training set and test set, respectively. Each pet photo is labeled with the value of 1 (Yes) or 0 (No) for each of the following features:
> 
> - Focus - Pet stands out against uncluttered background, not too close / far.
> - Eyes - Both eyes are facing front or near-front, with at least 1 eye / pupil decently clear.
> - Face - Decently clear face, facing front or near-front.
> Near - Single pet taking up significant portion of photo (roughly over 50% of photo width or height).
> - Action - Pet in the middle of an action (e.g., jumping).
> - Accessory - Accompanying physical or digital accessory / prop (i.e. toy, digital sticker), excluding collar and leash.
> - Group - More than 1 pet in the photo.
> - Collage - Digitally-retouched photo (i.e. with digital photo frame, combination of multiple photos).
> - Human - Human in the photo.
> - Occlusion - Specific undesirable objects blocking part of the pet (i.e. human, cage or fence). Note that not all blocking objects are considered occlusion.
> - Info - Custom-added text or labels (i.e. pet name, description).
> - Blur - Noticeably out of focus or noisy, especially for the pet’s eyes and face. For Blur entries, “Eyes” column is always set to 0.
>
>---

In [ ]:
# Efficient Data Types
dtype = {
    'Id': 'string',
    'Subject Focus': np.uint8, 'Eyes': np.uint8, 'Face': np.uint8, 'Near': np.uint8,
    'Action': np.uint8, 'Accessory': np.uint8, 'Group': np.uint8, 'Collage': np.uint8,
    'Human': np.uint8, 'Occlusion': np.uint8, 'Info': np.uint8, 'Blur': np.uint8,
    'Pawpularity': np.uint8,
}

train = pd.read_csv(CFG.TRAIN_CSV_PATH, dtype=dtype)
test = pd.read_csv(CFG.TEST_CSV_PATH, dtype=dtype)

<a id="tabular-exploration"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:lightgreen; border:0; color:white' role="tab" aria-controls="home"><center>EDA</center></h2>

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
profile_train = ProfileReport(train, title="Train Profiling Report")
profile_train.to_file("Train Profiling Report.html")
profile_train

In [ ]:
profile_test = ProfileReport(test, title="Test Profiling Report")
profile_test.to_file("Test Profiling Report.html")
# profile for 8 row is non informative
#profile_test

> ### **<span style="color:lightgreen;">Top Visualization for your Data</span>**
> AutoViz is an open-source visualization package under the AutoViML package library designed to automate many data scientists’ works. Many of the projects were quick and straightforward but undoubtedly helpful, including AutoViz.
> 
> AutoViz is a one-liner code visualization package that would automatically produce data visualization. Let’s try this package to show why AutoViz could help your work. First, we need to install AutoViz.


In [ ]:
AV = AutoViz_Class()
#Automatically produce dataset
AV.AutoViz("",dfte = train )

In [ ]:
print(f"Training Dataset Shape: {colored(train.shape, 'yellow')}")
print(f"Test Dataset Shape: {colored(test.shape, 'yellow')}")

In [ ]:
def get_image_file_path(df_type,image_id):
    return f'/kaggle/input/petfinder-pawpularity-score/{df_type}/{image_id}.jpg'

train['file_path'] = train['Id'].apply(lambda x: get_image_file_path('train',x))
test['file_path'] = test['Id'].apply(lambda x: get_image_file_path('test',x))

We notice the deviation at this QQPlot which seems to indicate a non-Gaussian distribution. We will check with the Kolmogorov-Smirnov test (Shapiro-Wilks is not suitable for a dataset greater than 5000 items).

In [ ]:
# Kolmogorov-Smirnov test with Scipy
stat, p = kstest(train['Pawpularity'],'norm')
print('Statistics=%.3f, p=%.3f' % (stat, p))

alpha = 0.05
if p > alpha:
    print(f'Sample looks Gaussian (fail to reject H0 at {int(alpha*100)}% test level)')
else:
    print(f'Sample does not look Gaussian (reject H0 at {int(alpha*100)}% test level)')

<a id="conclusion"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:lightgreen; border:0; color:white' role="tab" aria-controls="home"><center>Small conclusion about metadata</center></h2>
    
* Train/Test meta data have no missing values
* The pawpularity score is centered around 40 and has a peak on 0 and 100.
* The test clearly indicates that the distribution does not follow a Gaussian law. It will therefore be important to normalize the data according to the modeling chosen.
* We have pictures that meta table Eyes == 0 & Blur == 0, but actual picture has eyes (Look 👇 images)
* Correlation between: 
    1. "Eyes" and "Face";
    1. "Occlusion" and "Human";
    1. "Eyes" and "Blur"
    1. "Group" and "Near"

In [ ]:
plt.figure(figsize=(10, 10))
for i, img_path in enumerate(train[train.Eyes == 0].file_path.head(4)):
    ax = plt.subplot(2, 2, i+1)
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    plt.axis("off")
    plt.imshow(image)

In [ ]:
plt.figure(figsize=(10, 10))
for i, img_path in enumerate(train[(train.Eyes == 0) & (train.Blur == 0) ].file_path.head(4)):
    ax = plt.subplot(2, 2, i+1)
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    
    plt.axis("off")
    plt.imshow(image)

In [ ]:
feature_columns = ['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur']
y = train['Pawpularity']
train_features = train[feature_columns]
test_features = test[feature_columns]


x = train_features.to_numpy("float64")
y = y.to_numpy("float64")
test = test_features.to_numpy("float64")

In [ ]:
# Melt function
#Unpivot a DataFrame from wide to long format, optionally leaving identifiers set.
#This function is useful to massage a DataFrame into a format where one or more columns are identifier variables (id_vars),
#while all other columns, considered measured variables (value_vars), are “unpivoted” to the row axis, 
#leaving just two non-identifier columns, ‘variable’ and ‘value’.
df_train = train_features.melt(value_vars=feature_columns)
plt.figure(figsize = (15, 7))
sns.countplot(data=df_train, y="variable", hue="value" , palette =sns.color_palette(["#E4916C", "#BCE6EF"]))
plt.show()

<a id="distribution-plots"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:lightgreen; border:0; color:white' role="tab" aria-controls="home"><center> t-SNE | Clusters | Plots</center></h2>

## **<span style="color:lightgreen;">Clustering</span>**

In [ ]:
tsne = TSNE(n_components=2, random_state=CFG.SEED)
t_train_2 = tsne.fit_transform(train_features)
plt.figure(figsize=(15, 7))
sns.scatterplot(
    x=t_train_2[:, 0], y=t_train_2[:, 1],
    hue=train['Pawpularity'],
    alpha=0.3
)
plt.show()

In [ ]:
tsne = TSNE(n_components=3, random_state=CFG.SEED)
t_train_3 = tsne.fit_transform(train_features)
plt.figure(figsize=(15, 7))
sns.scatterplot(
    x=t_train_3[:, 0], y=t_train_3[:, 1],
    hue=train['Pawpularity'],
    alpha=0.3
)
plt.show()

## **<span style="color:lightgreen;">KMeans</span>**

In [ ]:
t_train_2 = pd.DataFrame(t_train_2, index=train.Id, columns=["c"+str(c) for c in range(2)])
km = KMeans(n_clusters=2, random_state=CFG.SEED).fit(t_train_2)
y_km = km.predict(t_train_2)
t_train_2["cluster"] = y_km
plt.figure(figsize=(15, 7))
sns.scatterplot(
    data=t_train_2,
    x="c0",
    y="c1",
    hue="cluster",
    alpha=0.3,
    palette=sns.color_palette(["#E4916C", "#BCE6EF"])
)
plt.show()

## **<span style="color:lightgreen;">PCA</span>**

In [ ]:
n_comp = 2
pca = PCA(n_components=n_comp, svd_solver='full', random_state=CFG.SEED)
X_pca = pca.fit_transform(train_features)
plt.figure(figsize=(15, 7))
sns.scatterplot(
    data=X_pca,
    x=X_pca[:, 0], y=X_pca[:, 1],
    hue=train['Pawpularity'],
    alpha=0.3
    )
plt.show()

<a id="tableswb"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:lightgreen; border:0; color:white' role="tab" aria-controls="home"><center> Tables in W&B</center></h2>

## **<span style="color:lightgreen;">Visualize Dataset Interactively using W&B Tables</span>**

It only requires 5 lines of extra code to get the power of W&B Tables.


1. Create a wandb.Table object. Imagine this to be an empty Pandas Dataframe.
1. Iterate through each row of the train.csv file and add_data to the wandb.Table object. Imagine this to be appending new rows to your Dataframe.
1. Log the W&B Tables using wandb.log API. You will use this API to log almost anything to W&B.
1. In a Juypter like interactive session, you need to call wandb.finish to close the initialized W&B run.



In [ ]:
columns = train.columns.tolist()
data_at = wandb.Table(columns=columns) # W&B Code 1

for i in tqdm(range(len(train))):# W&B Code 2
    row = train.loc[i]
    img_id = row.Id
    data_at.add_data(img_id, wandb.Image(f'/kaggle/input/petfinder-pawpularity-score/train/{img_id}.jpg'),*tuple(row.values[2:])) # W&B Code 3

wandb.log({'Raw Petfinder data': data_at}) # W&B Code 4
wandb.finish() # W&B Code 5

In [ ]:
# This is just to display the W&B run page in this interactive session.
from IPython import display

# we create an IFrame and set the width and height
iF = display.IFrame(run.url, width=1080, height=720)
iF

<a id="eof"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:lightgreen; border:0; color:white' role="tab" aria-controls="home"><center>work in progress</center></h2>